In [1]:
import os
import random
from time import time
from glob import glob
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from collections import Counter
import dill as pickle
from math import ceil
import dill as pickle


In [2]:
path = "/data2/ntua/data/planet/planet"
path_train = os.path.join(path, "train-jpg")
path_test = os.path.join(path, "test-jpg")
print(
    f"train files: {len(os.listdir(path_train))}, "
    f"test files: {len(os.listdir(path_test))}"
)

train files: 40479, test files: 40669


In [3]:
path_class = os.path.join(path, "train_classes.csv")
df_class = pd.read_csv(path_class)
#df_class = df_class.sample(n=2000) # limit images
print(df_class.shape)
print(df_class.head())

(40479, 2)
  image_name                                       tags
0    train_0                               haze primary
1    train_1            agriculture clear primary water
2    train_2                              clear primary
3    train_3                              clear primary
4    train_4  agriculture clear habitation primary road


In [4]:
df_class

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road
5,train_5,haze primary water
6,train_6,agriculture clear cultivation primary water
7,train_7,haze primary
8,train_8,agriculture clear cultivation primary
9,train_9,agriculture clear cultivation primary road


In [5]:
df_class["list_tags"] = df_class.tags.str.split(" ")
row_tags = df_class.list_tags.values
tags = [tag for row in row_tags for tag in row]
counter_tags = Counter(tags)

In [6]:
total_images = sum(counter_tags.values())

In [7]:
total_images

116278

In [8]:
df_dict={}
for key in counter_tags:
    df_dict[key] = df_class[df_class['list_tags'].apply(lambda x: key in x)]

In [9]:
df_dict

{'haze':         image_name                                  tags  \
 0          train_0                          haze primary   
 5          train_5                    haze primary water   
 7          train_7                          haze primary   
 43        train_43                      bare_ground haze   
 60        train_60                          haze primary   
 104      train_104                          haze primary   
 132      train_132                          haze primary   
 133      train_133                          haze primary   
 136      train_136                          haze primary   
 152      train_152                          haze primary   
 180      train_180                          haze primary   
 196      train_196                          haze primary   
 215      train_215                    haze primary water   
 228      train_228   agriculture haze primary road water   
 241      train_241         agriculture haze primary road   
 248      train_

In [10]:
number_of_images = 10000
number_of_samples_per_category = int(number_of_images/len(df_dict.keys()))+4

In [11]:
number_of_samples_per_category

592

In [12]:
number_of_samples_per_category*len(df_dict.keys())

10064

In [14]:
df_dict['partly_cloudy'].image_name.count()

7261

In [15]:
dataset_2000=pd.DataFrame()
images = []
for i,key in enumerate(df_dict):
    print(i,key)
    if i == 0:
        df_class = df_dict[key].sample(n=ceil(10000*len(df_dict[key].index)/total_images))
        print(df_class.image_name.count())
        images = list(df_class.image_name)
        dataset_2000 = pd.concat([dataset_2000, df_class])
    else:
        df_class = df_dict[key][~df_dict[key].image_name.isin(images)].sample(n=ceil(10000*len(df_dict[key].index)/total_images))
        print('here_:',df_class.image_name.count())
        new_im = list(df_class.image_name)
        images.extend(new_im)
        dataset_2000 = pd.concat([dataset_2000, df_class])

0 haze
232
1 primary
here_: 3227
2 agriculture
here_: 1060
3 clear
here_: 2446
4 water
here_: 638
5 habitation
here_: 315
6 road
here_: 695
7 cultivation
here_: 386
8 slash_burn
here_: 18
9 cloudy
here_: 180
10 partly_cloudy
here_: 625
11 conventional_mine
here_: 9
12 bare_ground
here_: 75
13 artisinal_mine
here_: 30
14 blooming
here_: 29
15 selective_logging
here_: 30
16 blow_down
here_: 9


In [16]:
dataset_2000.image_name.nunique()

10004

In [17]:
len(dataset_2000.index)

10004

In [18]:
len(images)

10004

In [19]:
row_tags2 = dataset_2000.list_tags.values
tags2 = [tag for row in row_tags for tag in row]
counter_tags2 = Counter(tags)

In [20]:
counter_tags

Counter({'haze': 2697,
         'primary': 37513,
         'agriculture': 12315,
         'clear': 28431,
         'water': 7411,
         'habitation': 3660,
         'road': 8071,
         'cultivation': 4547,
         'slash_burn': 209,
         'cloudy': 2089,
         'partly_cloudy': 7261,
         'conventional_mine': 100,
         'bare_ground': 862,
         'artisinal_mine': 339,
         'blooming': 332,
         'selective_logging': 340,
         'blow_down': 101})

In [21]:
dataset_2000.to_csv('/data2/ntua/data/planet/planet/train_10000.csv')

In [22]:
train = pd.read_csv('/data2/ntua/data/planet/planet/train_10000.csv')

In [23]:
images = list(train.image_name)

In [24]:
len(images)

10004

In [25]:
test_2000=pd.DataFrame()
for i,key in enumerate(df_dict):
    print(i,key)
    if i == 0:
        df_class = df_dict[key][~df_dict[key].image_name.isin(images)].sample(n=ceil(10000*len(df_dict[key].index)/total_images))
        print(df_class.image_name.count())
        images.extend(df_class.image_name)
        test_2000 = pd.concat([test_2000, df_class])
    else:
        df_class = df_dict[key][~df_dict[key].image_name.isin(images)].sample(n=ceil(10000*len(df_dict[key].index)/total_images))
        print('here_:',df_class.image_name.count())
        new_im = list(df_class.image_name)
        images.extend(df_class.image_name)
        test_2000 = pd.concat([test_2000, df_class])

0 haze
232
1 primary
here_: 3227
2 agriculture
here_: 1060
3 clear
here_: 2446
4 water
here_: 638
5 habitation
here_: 315
6 road
here_: 695
7 cultivation
here_: 386
8 slash_burn
here_: 18
9 cloudy
here_: 180
10 partly_cloudy
here_: 625
11 conventional_mine
here_: 9
12 bare_ground
here_: 75
13 artisinal_mine
here_: 30
14 blooming
here_: 29
15 selective_logging
here_: 30
16 blow_down
here_: 9


In [26]:
len(test_2000.index)

10004

In [27]:
test_2000.image_name.nunique()

10004

In [28]:
test_2000.to_csv('/data2/ntua/data/planet/planet/test_10000.csv')

# Evaluation

In [3]:
train_results = pickle.load(open("train_results_fold0.pkl", "rb"))

In [7]:
train_results['Y_thresh_val'].shape

(401, 17)

In [12]:
train_results['Y_hat_val'].shape

(401, 17)

In [13]:
train_results['Y_val'].shape

(401, 17)

In [11]:
train_results.keys()

dict_keys(['loss_train', 'loss_val', 'score_train', 'score_val', 'Y_hat_val', 'Y_thresh_val', 'Y_val'])

In [29]:
test_2000.image_name.unique()

array(['train_27343', 'train_3561', 'train_17765', ..., 'train_13856',
       'train_703', 'train_10374'], dtype=object)

In [33]:
train[train.image_name.isin(test_2000)]

,Unnamed: 0,image_name,tags,list_tags
